# Business Understanding 

 CMS rates hospitals in the US on a scale of 1-5 with the objective to make it easier for patients and consumers to compare the quality of hospitals.
 The ratings directly influence the choice of the hospital made by consumers and may have a significant impact on the revenue earned by hospitals.
 Thus, it is extremely important for hospitals to understand the methodology used by CMS for calculating the ratings so that they can work on
 improving the factors that influence them.
 This project is focused on developing an approach to calculate hospital ratings and using it to identify areas of improvement for
 certain hospitals. It will also require a thorough understanding of the rating system developed by CMS.


# Business Problem #

# The aim of analysis is to understand the methodology used by CMS for calculating the ratings and identify
# the factors influencing the ratings for hospitals, so that they can work on improving the factors that influence them.
# Recommend ways to improve the rating for Evanston Hospital to improve their current star rating of 3/5*


